<a href="https://colab.research.google.com/github/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/Tutorials/Tutorial_for_Neural_Network_Verification_with_Marabou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial for Neural Network Verification with Marabou

---

*As an example we try to verifiy the **robustness** of a classification Network trained on the MNIST dataset using the [Marabou library](https://github.com/NeuralNetworkVerification/Marabou).*

[Marabou documenation](https://neuralnetworkverification.github.io/Marabou/)

---

## Neural Network Training

In [41]:
%%capture

!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/DDiekmann/Applied-Verification-Lab-Neural-Networks/main/lib/mnist_trainer.py'}

In [47]:
import mnist_trainer

First, we will train a neural network with the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset

In [31]:
import torch
from torch import nn

import torch.onnx

In [32]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, number_of_neurons):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, output_dim),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [33]:
%%capture
train_dataloader, test_dataloader = mnist_trainer.load_dataset(batch_size=64)

In [48]:
model = mnist_trainer.train_model(
    NeuralNetwork(input_dim=28*28, output_dim=10, number_of_neurons=20), 
    epochs=3, 
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    )

Epoch 1
-------------------------------


100%|██████████| 938/938 [00:05<00:00, 162.66it/s]


Test Error: 
 Accuracy: 91.0%, Avg loss: 0.309950 

Epoch 2
-------------------------------


100%|██████████| 938/938 [00:05<00:00, 162.13it/s]


Test Error: 
 Accuracy: 92.6%, Avg loss: 0.248237 

Epoch 3
-------------------------------


100%|██████████| 938/938 [00:05<00:00, 159.48it/s]


Test Error: 
 Accuracy: 93.2%, Avg loss: 0.219686 

Done!


## Export the neural network

In [27]:
%pip install onnx onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
model_filename = "mnist_net.onnx"

# set model to eval mode
model.eval()

# create a dummy input in the shape of the input values
dummy_input = torch.randn(1, 28, 28)

torch.onnx.export(model,
                  dummy_input,
                  model_filename,
                  export_params=True,
                  verbose=False,
                  input_names=['image'],
                  output_names=['classification'],
                  )

## Installing Marabou

In [1]:
!git clone https://github.com/NeuralNetworkVerification/Marabou.git

Cloning into 'Marabou'...
remote: Enumerating objects: 25698, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 25698 (delta 4), reused 1 (delta 0), pack-reused 25686
Receiving objects: 100% (25698/25698), 208.52 MiB | 29.27 MiB/s, done.
Resolving deltas: 100% (18527/18527), done.


The following script builds and installs marabou. It takes about 20 minutes in colab.

In [16]:
!pwd
%cd Marabou
!mkdir build
%cd build
!cmake .. -DBUILD_PYTHON=ON
!cmake --build .
%cd /content
!pwd

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
dgelss.f:355:31:

                   LWORK_DORGBR=DUM(1)
                               1
dgelss.f:444:0:

          IF( M.GE.MNTHR ) THEN
 
dgeqlf.f:158:0:

      $                   MU, NB, NBMIN, NU, NX
 
dgerqf.f:158:0:

      $                   MU, NB, NBMIN, NU, NX
 
dgesvd.f:3472:0:

          IF( IE.LT.2 ) THEN
 
dgesvd.f:240:0:

      $                   NRVT, WRKBL
 
dgesvd.f:677:0:

          IF( M.GE.MNTHR ) THEN
 
dgesvd.f:237:0:

       INTEGER            BDSPAC, BLK, CHUNK, I, IE, IERR, IR, ISCL,
 
dgesvdx.f:670:0:

          IF( N.GE.MNTHR ) THEN
 
dgesvx.f:441:0:

                ROWCND = MAX( RCMIN, SMLNUM ) / MIN( RCMAX, BIGNUM )
 
dgesvx.f:441:0: Warning: ‘bignum’ may be used uninitialized in this function [-Wmaybe-uninitialized]
dgetc2.f:222:0:

          A( N, N ) = SMIN
 
dgetc2.f:204:0:

      $      CALL DSWAP( N, A( 1, JPV ), 1, A( 1, I ), 1 )
 
dgetc2.f:198:0:

      $      CALL DSWAP( N, A( 

In [17]:
import sys
sys.path.insert(1, '/content/Marabou')

https://neuralnetworkverification.github.io/Marabou/Examples/2_ONNXExample.html

In [58]:
from maraboupy import Marabou
import numpy as np

In [52]:
options = Marabou.createOptions(verbosity = 0)

In [59]:
network = Marabou.read_onnx(model_filename)